## Gather Data

In [98]:
import requests, io
import pandas as pd
import zipfile
from gurobipy import Model, GRB, quicksum
from IPython.display import clear_output

ridership_df = pd.DataFrame()

links = [
    'https://www.bart.gov/sites/default/files/2025-11/Ridership_202510.xlsx',
    'https://www.bart.gov/sites/default/files/2025-10/Ridership_202509.xlsx',
    'https://www.bart.gov/sites/default/files/2025-10/Ridership_202508.xlsx',
    'https://www.bart.gov/sites/default/files/2025-08/Ridership_202507.xlsx',
    'https://www.bart.gov/sites/default/files/2025-07/Ridership_202506.xlsx',
    'https://www.bart.gov/sites/default/files/2025-06/Ridership_202505.xlsx',
    'https://www.bart.gov/sites/default/files/2025-05/Ridership_202504.xlsx',
    'https://www.bart.gov/sites/default/files/2025-05/Ridership_202503.xlsx',
    'https://www.bart.gov/sites/default/files/2025-03/Ridership_202502.xlsx',
    'https://www.bart.gov/sites/default/files/2025-02/Ridership_202501.xlsx'
]

def extract_date_from_link(link):
    """
    Extract substring between 'Ridership_' and '.xlsx'.
    Example: 'Ridership_202510.xlsx' → '202510'
    """
    start = link.find("Ridership_") + len("Ridership_")
    end = link.find(".xlsx")
    return link[start:end]

def set_header_row(df, keyword="Exit Station Two-Letter Code"):
    """
    Find the row containing keyword and use it as the column headers.
    Return the cleaned DataFrame.
    """
    # locate the header row index
    header_idx = df.index[df.apply(lambda row: row.astype(str).str.contains(keyword).any(), axis=1)]
    
    if len(header_idx) == 0:
        # no header row found — return unchanged
        return df
    
    header_idx = header_idx[0]

    # set the header
    new_header = df.iloc[header_idx]
    df = df[header_idx + 1 :]  # drop the header row and all above it
    df.columns = new_header

    return df.reset_index(drop=True)


for link in links:
    response = requests.get(link)

    df = pd.read_excel(
        io.BytesIO(response.content),
        engine='openpyxl',
        header=None,                    
        sheet_name='Average Weekday'
    )

    df = set_header_row(df)

    # Add extracted date
    df["date"] = extract_date_from_link(link)            

    ridership_df = pd.concat([ridership_df, df], ignore_index=True)


In [99]:
# https://www.bart.gov/sites/default/files/docs/station-names.xls

station_map_df = pd.DataFrame({
    "code": [
        "RM","EN","EP","NB","BK","AS","MA","19","12","LM","FV","CL","SL","BF","HY","SH",
        "UC","FM","CN","PH","WC","LF","OR","RR","OW","EM","MT","PL","CC","16","24","GP",
        "BP","DC","CM","CV","ED","NC","WP","SS","SB","SO","MB","WD","OA","WS","AN","PC",
        "ML","BE"
    ],
    "stop_name": [
        "Richmond","El Cerrito Del Norte","El Cerrito Plaza","North Berkeley",
        "Downtown Berkeley","Ashby","MacArthur","19th Street Oakland",
        "12th Street / Oakland City Center","Lake Merritt","Fruitvale","Coliseum - OAC",
        "San Leandro","Bay Fair","Hayward","South Hayward","Union City","Fremont",
        "Concord","Pleasant Hill / Contra Costa Centre","Walnut Creek","Lafayette","Orinda","Rockridge",
        "West Oakland","Embarcadero","Montgomery Street","Powell Street",
        "Civic Center / UN Plaza","16th Street / Mission","24th Street / Mission","Glen Park",
        "Balboa Park","Daly City","Colma","Castro Valley","Dublin / Pleasanton",
        "North Concord / Martinez","Pittsburg / Bay Point","South San Francisco","San Bruno",
        "San Francisco International Airport","Millbrae (Caltrain Transfer Platform)","West Dublin / Pleasanton",
        "Oakland International Airport Station","Warm Springs / South Fremont","Antioch","Pittsburg Center",
        "Milpitas","Berryessa / North San Jose"
    ]
})

In [100]:
GTFS_URL = "https://www.bart.gov/dev/schedules/google_transit.zip"

response = requests.get(GTFS_URL)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Load GTFS components into DataFrames
routes = pd.read_csv(z.open("routes.txt"))
stops = pd.read_csv(z.open("stops.txt"))
trips = pd.read_csv(z.open("trips.txt"), dtype={"route_id": str})
stop_times = pd.read_csv(z.open("stop_times.txt"))
calendar = pd.read_csv(z.open("calendar.txt"))

# Join trips with routes
trip_routes = trips.merge(routes, on="route_id", how="left")

# Join stop_times with stops
times_with_stops = stop_times.merge(stops, on="stop_id", how="left")

# Final joined table
full_schedule = (
    trip_routes
    .merge(times_with_stops, on="trip_id", how="left")
    .sort_values(["route_id", "trip_id", "stop_sequence"])
)

full_schedule['stop_name'] = full_schedule['stop_name'].str.replace("", "", regex=False)

mapped_schedule = full_schedule.merge(station_map_df, left_on="stop_name", right_on="stop_name", how="left")

# mapped_schedule.to_excel("C:/Users/chhri/Downloads/bart_full_schedule.xlsx")

In [118]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum

stops_df = mapped_schedule.copy()

def parse_gtfs_time(t):
    """
    Parse time strings like '06:11:00' or '24:05:00' into minutes since midnight.
    GTFS allows hours >= 24, meaning times past midnight but same service day.
    """
    if pd.isna(t):
        return np.nan
    hh, mm, ss = map(int, str(t).split(":"))
    return hh * 60 + mm + ss / 60.0   # minutes since midnight (possibly > 24*60)

def compute_travel_times(stops_df):
    df = stops_df.copy()

    # parse times into numeric minutes
    df["arr_min"] = df["arrival_time"].apply(parse_gtfs_time)
    df["dep_min"] = df["departure_time"].apply(parse_gtfs_time)

    # sort properly
    df = df.sort_values(["route_id", "trip_id", "stop_sequence"])

    arc_rows = []

    for (route_id, trip_id), group in df.groupby(["route_id", "trip_id"]):
        group = group.sort_values("stop_sequence")

        dep_times = group["dep_min"].values
        arr_times = group["arr_min"].values
        seq      = group["stop_sequence"].values
        codes    = group["code"].values
        dep_str  = group["departure_time"].values
        arr_str  = group["arrival_time"].values

        for i in range(len(group) - 1):
            travel = arr_times[i+1] - dep_times[i]   # minutes
            travel_min = int(round(travel))

            arc_rows.append({
                "route_id": route_id,
                "trip_id": trip_id,
                "from_stop": codes[i],
                "to_stop": codes[i+1],
                "from_seq": seq[i],
                "to_seq": seq[i+1],
                "dep_time_str": dep_str[i],
                "arr_time_str": arr_str[i+1],
                "travel_time_min": travel_min
            })

    return pd.DataFrame(arc_rows)

arc_df = compute_travel_times(stops_df)

def build_nodes(arc_df):
    # each "to" event defines a station–time node
    nodes = arc_df["to_stop"] + "_" + arc_df["arr_time_str"]
    V = list(nodes.unique())
    return V

V = build_nodes(arc_df)

def build_stage_arcs(arc_df):
    A_stage = []
    for row in arc_df.itertuples(index=False):
        arc_name = f"{row.from_stop}_{row.dep_time_str}__{row.to_stop}_{row.arr_time_str}"
        A_stage.append({
            "arc": arc_name,
            "from_node": f"{row.from_stop}_{row.dep_time_str}",
            "to_node":   f"{row.to_stop}_{row.arr_time_str}",
            "travel_time": row.travel_time_min,
            "origin_code": row.from_stop,
            "dest_code":   row.to_stop,
            "trip_id":     row.trip_id
        })
    return A_stage

A_stage = build_stage_arcs(arc_df)

# index ridership by origin
ridership_df_idx = ridership_df.set_index("Exit Station Two-Letter Code")

def build_projected_ridership_dict(ridership_df_idx, growth_rate=1.0):
    """
    Returns a dict: projected_lookup[(origin, dest)] = scalar ridership.
    Applies an optional growth factor.
    """
    clean_index = ridership_df_idx.index
    clean_index = clean_index[~clean_index.isin(["Grand Total"])]
    clean_index = clean_index.dropna()

    clean_cols = ridership_df_idx.columns
    clean_cols = clean_cols[~clean_cols.isin(["Grand Total", "date"])]
    clean_cols = clean_cols.dropna()

    projected_lookup = {}

    for origin in clean_index:
        for dest in clean_cols:
            vals = ridership_df_idx.loc[origin, dest]

            if isinstance(vals, pd.Series):
                vals = vals.dropna()
                avg_ridership = vals.mean()
            else:
                avg_ridership = float(vals) if pd.notna(vals) else 0.0

            projected = avg_ridership * growth_rate
            projected_lookup[(origin, dest)] = float(projected)

    return projected_lookup

projected_lookup = build_projected_ridership_dict(ridership_df_idx, growth_rate=1.0)

def compute_demand_for_arcs(A_stage, projected_lookup, stops_df):
    """
    Compute scalar demand d_a for each arc using projected OD demand.
    d_a = sum of riders who board at origin and exit at any downstream stop
          along that trip.
    """
    # Precompute stop order for each trip
    stop_orders = {}
    for trip_id, group in stops_df.groupby("trip_id"):
        group_sorted = group.sort_values("stop_sequence")
        stop_orders[trip_id] = list(group_sorted["code"])

    demand_dict = {}

    for arc in A_stage:
        trip_id = arc["trip_id"]
        origin = arc["origin_code"]
        dest   = arc["dest_code"]

        stops = stop_orders[trip_id]
        idx_origin = stops.index(origin)
        idx_dest   = stops.index(dest)

        downstream = stops[idx_dest:]   # dest and all following stops

        d_sum = 0.0
        for down_station in downstream:
            if not isinstance(down_station, str):
                continue
            key = (origin, down_station)
            d_sum += projected_lookup.get(key, 0.0)

        demand_dict[arc["arc"]] = float(d_sum)

    return demand_dict

d = compute_demand_for_arcs(A_stage, projected_lookup, stops_df)

# Remove duplicates based on the 'arc' field
seen = set()
A_stage_unique = []

for a in A_stage:
    if a["arc"] not in seen:
        A_stage_unique.append(a)
        seen.add(a["arc"])

In [119]:

model = Model("Bart_Fleet_Optimization")

# t[a] = integer number of train units assigned to arc a
t = model.addVars(
    [arc["arc"] for arc in A_stage_unique],
    vtype=GRB.INTEGER,
    lb=0,
    name="t"
)

model.setObjective(
    quicksum(arc["travel_time"] * t[arc["arc"]] for arc in A_stage_unique),
    GRB.MINIMIZE
)

# fleet-style objective (unweighted)
model.setObjective(
    quicksum(t[arc["arc"]] for arc in A_stage_unique),
    GRB.MINIMIZE
)

m_seats_per_unit = 163  # seats per car / unit
n_max_units      = 5    # max units per train

for arc in A_stage_unique:
    name = arc["arc"]
    model.addConstr(t[name] <= n_max_units, name=f"cap_{name}")

for arc in A_stage_unique:
    name = arc["arc"]
    model.addConstr(
        m_seats_per_unit * t[name] >= d[name],
        name=f"demand_{name}"
    )

model.optimize()

solution = {arc["arc"]: t[arc["arc"]].X for arc in A_stage_unique}

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 29038 rows, 14519 columns and 29038 nonzeros
Model fingerprint: 0xc8b0a832
Variable types: 0 continuous, 14519 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 7e+03]
Presolve removed 343 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'X'

In [117]:
arcs = [a["arc"] for a in A_stage]
unique_arcs = set(arcs)

print(len(arcs), len(unique_arcs))

52646 14519
